In [1]:
import WebNLG_xmlReader.benchmark_reader as xml_reader
import os.path
import pickle

# Dataset Cleaning and Preperations

CACAPO contains data for both pipeline and neural end-to-end structures. As this project only focuses on E2E models, we will not need a majority of the data. The code below extracts the data and makes it easier to retrieve for model fine-tuning

In [2]:
combined_train_dataset = [
                '../Data/CACAPO/en/Incidents/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/en/Sports/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/en/Stocks/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/en/Weather/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/nl/Incidents/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/nl/Sports/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/nl/Stocks/WebNLGFormatTrain.xml', 
                '../Data/CACAPO/nl/Weather/WebNLGFormatTrain.xml']

combined_dev_dataset = [
                '../Data/CACAPO/en/Incidents/WebNLGFormatDev.xml', 
                '../Data/CACAPO/en/Sports/WebNLGFormatDev.xml', 
                '../Data/CACAPO/en/Stocks//WebNLGFormatDev.xml', 
                '../Data/CACAPO/en/Weather/WebNLGFormatDev.xml', 
                '../Data/CACAPO/nl/Incidents/WebNLGFormatDev.xml', 
                '../Data/CACAPO/nl/Sports/WebNLGFormatDev.xml', 
                '../Data/CACAPO/nl/Stocks/WebNLGFormatDev.xml', 
                '../Data/CACAPO/nl/Weather/WebNLGFormatDev.xml']

combined_test_dataset = [
                '../Data/CACAPO/en/Incidents/WebNLGFormatTest.xml', 
                '../Data/CACAPO/en/Sports/WebNLGFormatTest.xml', 
                '../Data/CACAPO/en/Stocks/WebNLGFormatTest.xml', 
                '../Data/CACAPO/en/Weather/WebNLGFormatTest.xml', 
                '../Data/CACAPO/nl/Incidents/WebNLGFormatTest.xml', 
                '../Data/CACAPO/nl/Sports/WebNLGFormatTest.xml', 
                '../Data/CACAPO/nl/Stocks/WebNLGFormatTest.xml', 
                '../Data/CACAPO/nl/Weather/WebNLGFormatTest.xml']

all_data = [combined_train_dataset, combined_dev_dataset, combined_test_dataset]

In [61]:

# create instance of benchmark class to transform xml
train_instance =  xml_reader.Benchmark()
dev_instance =  xml_reader.Benchmark()
test_instance =  xml_reader.Benchmark()



def transform_data(data):
    """
    This function calls the xml_reader code to transform the xml into a more suitable code version to use.
    The function takes in the 3 different datasets, which it then transforms and collects into usable variables.
    """
    #loop through the different datasets groups and transform the xml into usable code
    for datasets in data:
        #choose the right files
        files = xml_reader.select_files(datasets)
        
        #For each datasplit, transform the xml and store the transformation into a usable variable
        try:
            if datasets == combined_train_dataset:
                train_instance.fill_benchmark(files)
            elif datasets == combined_dev_dataset:
                dev_instance.fill_benchmark(files)
            elif datasets == combined_test_dataset:
                test_instance.fill_benchmark(files)
            
            print(f'Completed the transformation of the datasets \n')
        except:
            print("Error: The proper datasets have not been found. Please check that all dataset splits are available")
    
    return train_instance, dev_instance, test_instance


def total_data_check(data_instance, iteration):
    labels = ['Train', 'Dev', 'Test']
    print(f"Number of entries: in {labels[iteration]}:      {data_instance.entry_count()} ") 
    print(f"Number of texts: in {labels[iteration]}:      {data_instance.total_lexcount()} ")
    print(f"Number of distinct properties in {labels[iteration]}:      {len(list(data_instance.unique_p_mtriples()))}")
    print("\n")

    
def single_entry_check(data_instance):
    
    for entry in data_instance.entries:
        #print(f'entry.id        {entry.id}')
        if entry.id == 'Id1':
            print(f"Info about {entry.id} in category '{entry.category}' in size '{entry.size}':")
            print("# of lexicalisations", entry.count_lexs())
            print("Properties: ", entry.relations())
            print("RDF triples: ", entry.list_triples())
            print("Subject:", entry.modifiedtripleset.triples[0].s)
            print("Predicate:", entry.modifiedtripleset.triples[0].p)
            print("Lexicalisation:", entry.lexs[0].lex)
            #print("Another lexicalisation:", entry.lexs[1].lex)
            if entry.dbpedialinks:
                # dbpedialinks is a list where each element is a Triple instance
                print("DB link, en:", entry.dbpedialinks[0].s)  # subject in English

            print("Article text", entry.lexs[0].return_text()) 
            

def extract_data(data_instance):
    RDF_set, text_set = [], []

    for entry in data_instance.entries:
        #RDF_text_set.append((entry.list_triples(), entry.lexs[0].return_text()))

        RDF_set.append(entry.list_triples())
        text_set.append(entry.lexs[0].return_text())


    #print(len(RDF_text_set))
    return RDF_set, text_set 

def write_to_file(data, iteration, data_type):
    """
    Data = dataset
    Iteration = iteration to determine the dataset split
    data_type = is the dataset RDF or text
    """
    labels = ['Train', 'Dev', 'Test']

    #print(f"path check  {os.path.exists(f'Data/Cleaned_data/{labels[iteration]}')}")

    try:

        if (os.path.exists(f'Data/Cleaned_data/{labels[iteration]}') == False) :
            print("Entered path check")

            save_path = 'C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/'

            name_of_file = f'{labels[iteration]}_{data_type}'

            completeName = os.path.join(save_path, name_of_file+".pkl")         

            with open(completeName, 'wb') as fp:
                print(f"Entered pickle check         {completeName}")

                pickle.dump(data, fp)

    except:
        print(f'file for {labels[iteration]} already exists')


def retrieve_data(file_name):
    
    dataset_path = f"../Data/Cleaned_data/{file_name}.pkl"
    with open(dataset_path, 'rb') as f:
        dataset = pickle.load(f)
    
    return dataset

def Overal_function(data):
    transformed_train, transformed_dev, transformed_test = transform_data(data)
    # print(f'train:      {transformed_train}')
    # print(f'dev:      {transformed_dev}')
    # print(f'test:      {transformed_test}')

    combined_transformation = [transformed_train, transformed_dev, transformed_test]
    RDF_text_datasets =[]

    for iteration, dataset in enumerate(combined_transformation):
        
             
        # print(f'dataset {dataset}')
        # print(f'iteration {iteration}')
        
        #total_data_check(dataset, iteration)
        #single_entry_check(dataset)

        RDF_set, Text_set = extract_data(dataset)
        #RDF_text_datasets.append()
        write_to_file(RDF_set, iteration, 'RDF')
        write_to_file(Text_set, iteration, 'text')
        print("\n\n\n")

    #return RDF_text_datasets

In [62]:
Overal_function(all_data)

Completed the transformation of the datasets 

Completed the transformation of the datasets 

Completed the transformation of the datasets 

0
Entered path check
Entered pickle check         C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/Train_RDF.pkl
0
Entered path check
Entered pickle check         C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/Train_text.pkl




1
Entered path check
Entered pickle check         C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/Dev_RDF.pkl
1
Entered path check
Entered pickle check         C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/Dev_text.pkl




2
Entered path check
Entered pickle check         C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/Test_RDF.pkl
2
Entered path check
Entered pickle check         C:/Users/Simon/Desktop/Arria Thesis/MscThesis/Data/Cleaned_data/Test_text.pkl






# Transformer modelling

In [3]:
!pip install sentencepiece

In [109]:
def tokenize_data(data, tokenizer):
    """
    Data is a list of tuples consisting of (RDF_triple, article text)
    """
    tokenized_data = []
    for entry in data:
        #tokenized_data.append(tokenizer(entry, return_tensors="tf", padding=True))
        entry = tokenizer(entry, return_tensors="tf", padding=True)
        
    return data
    # tokenized_data = tokenizer(data, return_tensors='tf', padding=True)
    # return tokenized_data


In [73]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-base")

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")

e:\Anaconda\envs\tensorflow_env\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the mode

In [110]:
import numpy as np

#get data
train_ds_RDF = retrieve_data('Train_RDF')
#np_train_RDF = np.asarray(train_ds_RDF)
tokenized_train_RDF = tokenize_data(train_ds_RDF, tokenizer)

In [120]:
for iteration, entry in enumerate(tokenized_train_RDF):
    if iteration == 0:
        tokenized_entry = tokenizer(entry)
        print(tokenized_entry["input_ids"])

[[5262, 25160, 1820, 18443, 834, 5515, 834, 77, 900, 1938, 1], [6220, 517, 3868, 1820, 388, 1], [6220, 567, 5937, 49, 1820, 80, 1], [6220, 134, 17, 144, 302, 1820, 16, 834, 1071, 7, 17, 9666, 1]]


In [123]:
for iteration, entry in enumerate(tokenized_train_RDF):
    if iteration == 0:
        print(entry)

['shootingType | deadly_home_invasion', 'suspectGender | man', 'suspectNumber | one', 'suspectStatus | in_custody']


In [83]:
tokenized_train_RDF

[{'input_ids': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
 array([[ 5262, 25160,  1820, 18443,   834,  5515,   834,    77,   900,
          1938,     1,     0,     0],
        [ 6220,   517,  3868,  1820,   388,     1,     0,     0,     0,
             0,     0,     0,     0],
        [ 6220,   567,  5937,    49,  1820,    80,     1,     0,     0,
             0,     0,     0,     0],
        [ 6220,   134,    17,   144,   302,  1820,    16,   834,  1071,
             7,    17,  9666,     1]])>, 'attention_mask': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>},
 {'input_ids': <tf.Tensor: shape=(4, 24), dtype=int32, numpy=
 array([[ 3125, 20773,  9377,  1820,   234,   834,    77,   834,   532,
           834,   948,  2915,   834, 10734,   834,   858,   834,   308,
          9197,    53,  

In [77]:
tokenized_train_RDF[0]

{'input_ids': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
array([[ 5262, 25160,  1820, 18443,   834,  5515,   834,    77,   900,
         1938,     1,     0,     0],
       [ 6220,   517,  3868,  1820,   388,     1,     0,     0,     0,
            0,     0,     0,     0],
       [ 6220,   567,  5937,    49,  1820,    80,     1,     0,     0,
            0,     0,     0,     0],
       [ 6220,   134,    17,   144,   302,  1820,    16,   834,  1071,
            7,    17,  9666,     1]])>, 'attention_mask': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [91]:
train_text = retrieve_data("Train_text", tokenizer)
np_train_text = np.array(train_text)
tokenized_train_text = tokenize_data(np_train_text)

In [94]:
len(tokenized_train_RDF)

15290

In [95]:
len(tokenized_train_text)

15290

In [100]:
np.array(tokenized_train_text)
print(f'train RDF = {np.shape(tokenized_train_RDF)}')
print(f'train text = {np.shape(tokenized_train_text)}')

train RDF = (15290, 2)
train text = (15290, 2)


In [85]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(3e-5))

model.fit(tokenized_train_RDF, tokenized_train_text)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


ValueError: Data cardinality is ambiguous:
  x sizes: 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 4, 4, 4, 4, 1, 1, 2, 2, 4, 4, 5, 5, 3, 3, 4, 4, 5, 5, 3, 3, 2, 2, 1, 1, 6, 6, 3, 3, 5, 5, 1, 1, 1, 1, 1, 1, 7, 7, 3, 3, 2, 2, 1, 1, 3, 3, 6, 6, 5, 5, 4, 4, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 4, 4, 2, 2, 3, 3, 2, 2, 3, 3, 5, 5, 2, 2, 4, 4, 3, 3, 1, 1, 7, 7, 2, 2, 1, 1, 3, 3, 3, 3, 2, 2, 1, 1, 5, 5, 3, 3, 2, 2, 4, 4, 2, 2, 2, 2, 3, 3, 5, 5, 4, 4, 1, 1, 1, 1, 2, 2, 1, 1, 3, 3, 2, 2, 1, 1, 1, 1, 5, 5, 4, 4, 3, 3, 3, 3, 5, 5, 2, 2, 3, 3, 1, 1, 5, 5, 3, 3, 5, 5, 6, 6, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 3, 3, 1, 1, 3, 3, 3, 3, 1, 1, 5, 5, 5, 5, 1, 1, 2, 2, 3, 3, 4, 4, 6, 6, 1, 1, 4, 4, 5, 5, 3, 3, 3, 3, 6, 6, 2, 2, 4, 4, 3, 3, 2, 2, 3, 3, 3, 3, 2, 2, 1, 1, 4, 4, 2, 2, 3, 3, 2, 2, 1, 1, 1, 1, 5, 5, 3, 3, 3, 3, 3, 3, 2, 2, 1, 1, 2, 2, 4, 4, 2, 2, 2, 2, 6, 6, 3, 3, 1, 1, 4, 4, 1, 1, 6, 6, 1, 1, 4, 4, 1, 1, 4, 4, 1, 1, 4, 4, 6, 6, 3, 3, 3, 3, 5, 5, 2, 2